Importing necessary modules 

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses
from sklearn.metrics import f1_score
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Loading the mnist classification data

In [2]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
x_train.shape

(60000, 28, 28)

Padding the dataset

In [3]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train.shape

TensorShape([60000, 32, 32])

Expanding the tensor dimensions

In [4]:
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train.shape

TensorShape([60000, 32, 32, 1])

Separating for validation set

In [5]:
x_val = x_train[-2000:,:,:,:] 
y_val = y_train[-2000:] 
x_train = x_train[:-2000,:,:,:] 
y_train = y_train[:-2000]

Data Augmentation

In [6]:
datagen = ImageDataGenerator(
    rotation_range=10,         
    width_shift_range=0.1,     
    height_shift_range=0.1,    
    shear_range=0.1,           
    zoom_range=0.1,            
    horizontal_flip=False,     
    fill_mode='nearest'  
)

datagen.fit(x_train)

Building the model by changing Activation function from Dying Relu to tanh 

In [7]:
model = models.Sequential()

model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=x_train.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))

model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))

model.add(layers.Conv2D(120, 5, activation='tanh'))

model.add(layers.Flatten())

model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

c:\Users\sneha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 14, 14, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

Compiling model with Adam optimizer for 40 epochs

In [8]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=40, validation_data=(x_val, y_val))

Epoch 1/40


c:\Users\sneha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 0.1054 - loss: 2.3258 - val_accuracy: 0.6665 - val_loss: 1.0671
Epoch 2/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 12s 14ms/step - accuracy: 0.5443 - loss: 1.3090 - val_accuracy: 0.8355 - val_loss: 0.5663
Epoch 3/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.6785 - loss: 0.9538 - val_accuracy: 0.9170 - val_loss: 0.3328
Epoch 4/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.7642 - loss: 0.7167 - val_accuracy: 0.9440 - val_loss: 0.2264
Epoch 5/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.8317 - loss: 0.5271 - val_accuracy: 0.9620 - val_loss: 0.1624
Epoch 6/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.8677 - loss: 0.4144 - val_accuracy: 0.9585 - val_loss: 0.1627
Epoch 7/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.8856 - loss: 0.3588 - val_accuracy: 0.9630 - val_loss: 0.1376
Epoch 8/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.8977 - loss: 0.3304 - val_accurac

Model's accuracy

In [10]:
model.evaluate(x_test, y_test)
print(f"Accuracy on test set: {model.evaluate(x_test, y_test)[1]:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9633 - loss: 0.1135
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9633 - loss: 0.1135
Accuracy on test set: 0.97


F1 SCORE

F1 Score Range: The F1 score ranges from 0 to 1:

In [11]:
predictions = model.predict(x_test)

predicted_classes = np.argmax(predictions, axis=1)

f1 = f1_score(y_test, predicted_classes, average='macro')

print(f"f1 score: {f1:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
f1 score: 0.97


After Data augmentation:

    The accuracy here is 0.98

    The F1 Score here is 0.98